# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [15]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try brew install
    xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-brew-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-brew packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages s

### Importing the libraries

In [25]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [26]:
class Network(nn.Module):

    def __init__(self, state_size, action_size, seed = 42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, state):
        x = self.fc1(state)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.fc3(x)


## Part 2 - Training the AI

### Setting up the environment

In [27]:
import gymnasium as gym

env = gym.make("LunarLander-v2")
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
n_actions = env.action_space.n

print(f"state shape {state_shape}")
print(f"state size {state_size}")
print(f"n actions {n_actions}")

state shape (8,)
state size 8
n actions 4


### Initializing the hyperparameters

In [28]:
lr = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [29]:
class ReplayMemory(object):
    
    def __init__(self, capacity):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.capacity = capacity
        self.memory = []
        
        
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
            
    def sample(self, batch_size):
        xps = random.sample(self.memory, k=batch_size)
        states = torch.from_numpy(np.vstack([xp[0] for xp in xps if xp is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([xp[1] for xp in xps if xp is not None])).long().to(self.device)
        rewards = torch.from_numpy(np.vstack([xp[2] for xp in xps if xp is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([xp[3] for xp in xps if xp is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([xp[4] for xp in xps if xp is not None]).astype(np.uint8)).float().to(self.device)
        
        return states, next_states, actions, rewards, dones



### Implementing the DQN class

In [36]:
class Agent():

    def __init__(self, state_size, action_size):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = Network(state_size, action_size).to(self.device)
        self.target_qnetwork = Network(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=lr)
        self.memory = ReplayMemory(replay_buffer_size)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0 and len(self.memory.memory) > minibatch_size:
            xps = self.memory.sample(minibatch_size)
            self.learn(xps, discount_factor)

    def act(self, state, epsilon=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()

        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, xps, discount_factor):
        states, next_states, actions, rewards, dones = xps
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

    def soft_update(self, local_model, target_model, interpolation_param):
        for local_param, target_param in zip(local_model.parameters(), target_model.parameters()):
            target_param.data.copy_(interpolation_param * local_param.data + (1.0 - interpolation_param) * target_param.data)


### Initializing the DQN agent

In [37]:
agent = Agent(state_size, action_size=n_actions)

### Training the DQN agent

In [39]:
n_episodes = 2_000
max_n_timesteps_per_episode = 1_000
epsilon_starting = 1.0
epsilon_ending = 0.01
epsilon_decay = 0.995
epsilon = epsilon_starting
scores_on_100_episodes = deque(maxlen=100)

for episode in range(1, n_episodes + 1):
    state, _ = env.reset()
    score = 0
    for t in range(max_n_timesteps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        
        state = next_state
        score += reward
        
        if done:
            break

    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_ending, epsilon * epsilon_decay)
    
    print('\rEpisode {}\tAverage score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")
    if episode % 100 == 0:
        print('\rEpisode {}\tAverage score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
        
    if np.mean(scores_on_100_episodes) >= 200.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
        torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
        break

Episode 100	Average score: -105.47
Episode 200	Average score: -28.643
Episode 300	Average score: 56.856
Episode 400	Average score: 187.80
Episode 408	Average score: 200.37
Environment solved in 308 episodes!	Average score: 200.37


## Part 3 - Visualizing the results

In [50]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

os.environ["IMAGEIO_FFMPEG_EXE"] = "/opt/homebrew/bin/ffmpeg"

def show_video_of_model(agent: Agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)
    
show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    video_list = glob.glob('*.mp4')
    if len(video_list) > 0:
        most_recent_vid = video_list[0]
        video = io.open(most_recent_vid, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay loop controls style="height:384px;"><source src="data:video/mp4;base64,{0}" /></video>'''.format(encoded.decode('ascii'))))
    else:
        print('Could not find video.')

show_video()
        
    

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
